###### print("mantap")

In [11]:
import cv2
import numpy as np
import bnn
from IPython.display import display
from IPython.display import clear_output

In [12]:
from PIL import Image as im
from PIL import ImageEnhance
from PIL import ImageOps
from PIL import ImageDraw

from pynq import Overlay
from pynq.lib.video import *

hw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW2A2,'cifar10',bnn.RUNTIME_HW)
sw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A2,'cifar10',bnn.RUNTIME_SW)


In [13]:
# Mode = VideoMode(640,480,24)
# base = Overlay('base.bit')
# hdmi_out = base.video.hdmi_out
# hdmi_out.configure(Mode)
# hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480



In [14]:
# videoIn = cv2.VideoCapture(0)
videoIn = cv2.VideoCapture("mantap.mp4")
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
videoIn.set(cv2.CAP_PROP_BUFFERSIZE, 1);

print("Capture device is open: " + str(videoIn.isOpened()))
while True:
    ret, frame_vga = videoIn.read()
    if ret:
        frame_vga = cv2.cvtColor(frame_vga,cv2.COLOR_BGR2RGB)
        img = im.fromarray(frame_vga)
        bright = ImageEnhance.Brightness(img)                                     
        img = bright.enhance(0.95)
        images = []
        bounds = []
        # 64,96
        for s in [64,96]:
            stride = s // 4
            x_tiles = img.width // stride
            y_tiles = img.height // stride
            for j in range(y_tiles):
                for i in range(x_tiles):
                    bound = (stride * i, stride * j, stride * i + s, stride * j + s)
                    if bound[2] <= img.width and bound[3] < img.height:
                        c = img.crop(bound)
                        images.append(c)
                        bounds.append(bound)
        results = hw_classifier.classify_images_details(images)
        result=result.reshape(len(images),10)
        draw = ImageDraw.Draw(im)
        i=0
        for image in images:
            if result[i][1] > 410:
                draw.rectangle(bounds[i], outline='red')
            i=i+1
        display(im)
videoIn.release()
    
# display(frame_vga)
    
#     print(type(im2))
    

Capture device is open: True
Inference took 5184238.00 microseconds, 1163.17 usec per image
Classification rate: 859.72 images per second


AttributeError: module 'PIL.Image' has no attribute 'load'

In [ ]:
videoIn.release()

In [ ]:
videoIn = cv2.VideoCapture("mantap.mp4")
object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    